In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as const
import matplotlib as mpl
from jupyterthemes import jtplot #These two lines can be skipped if you are not using jupyter themes
jtplot.reset()

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=67.4, Om0=0.314)
import scipy as sp
import multiprocessing as mp

from matplotlib.ticker import StrMethodFormatter
import time
start_total = time.time()

In [2]:
import os
my_path = '/home/tomi/Documentos/Fisica/Tesis/escrito-tesis/images/'

In [3]:
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LensModel.lens_model_extensions import LensModelExtensions
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver

In [4]:
zl = 0.2; zs = 1.2
Dl = cosmo.angular_diameter_distance(zl)   
Ds = cosmo.angular_diameter_distance(zs)    
Dls = cosmo.angular_diameter_distance_z1z2(zl, zs)
G = const.G
rho_crit = (cosmo.critical_density(zl)).to(u.kg/u.m**3)
c_light = (const.c).to(u.cm/u.second)

#r0 = 10*u.kpc
r0 = 10.0*u.kpc
#r0 = 0.1*u.kpc
pi = np.pi

def scale_radius(v,Dl,Ds,Dls):                               #this is e0 in eq 3.42 meneghetti, eq 1 barnacka 2014
    return (4.*pi*v**2/c_light**2*Dl*Dls/Ds).decompose()
def theta_E_SIS():
    'in arcsec'
    pre_theta_E = (scale_radius(v,Dl,Ds,Dls)/Dl).decompose()
    return pre_theta_E*u.rad.to('arcsec', equivalencies=u.dimensionless_angles()) 

v = 180 *u.km/u.s
ss_r = scale_radius(v,Dl,Ds,Dls) 
print('scale radius (m): ',ss_r)
print('scale radius (kpc): ',ss_r.to(u.kpc))
print('theta_E: ',theta_E_SIS() ,'arcsec')
theta_E_num = theta_E_SIS()
re = (const.e.esu**2/const.m_e/(c_light**2)).decompose()
print('Classic electron radius: ',re)

scale radius (m):  7.705329461274929e+19 m
scale radius (kpc):  2.49712721364453 kpc
theta_E:  0.7301786792241515 arcsec
Classic electron radius:  2.817940324670788e-15 m


In [5]:
nu = 130.0 * u.MHz
omega = nu*(1.+zl)*2*np.pi
n0 = 300./(u.cm)**(3)

In [6]:
theta_0 = (r0/Dl).decompose()*u.rad.to('arcsec', equivalencies=u.dimensionless_angles()) 
print('Angular scale radius (theta_0 = r0/Dl): ',theta_0, 'arcsec')

Angular scale radius (theta_0 = r0/Dl):  2.9240748137875747 arcsec


In [7]:
from lenstronomy.LensModel.Profiles.plasma_exponential_sie import PlasmaExponentialSIE
perfil = PlasmaExponentialSIE()

In [8]:
def image_position_3(x, y, eta):
    x_s, y_s = x, y
    x_pre, y_pre = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_lens_list, \
                    sourcePos_x=x_s, sourcePos_y=y_s, min_distance=0.05, search_window=5, \
                    precision_limit=10**(-8), num_iter_max=100,non_linear=True)
    print(x_pre)
    print(y_pre)
    mag = [0,0]
    t = [0,0]
    for k in range(len(x_pre)):
        mag[k] = lensModel.magnification(x_pre[k], y_pre[k], kwargs = kwargs_lens_list)
        phi = PlasmaExponentialSIE.function(perfil, x_pre[k], y_pre[k], theta_E_num.value, eta, A, B, C ,psi0_plasma.value, theta_0.value)
        t[k] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (x_s - x_pre[k] )**2 + (y_s - y_pre[k])**2) - phi**2)*(u.arcsec**2).to('rad**2')).to('s').value
    r = np.abs(mag[0]/mag[1])
    rr = [r,r]
    dt = np.abs(t[0] - t[1])/24/60/60 #convert seconds to days
    dt = [dt,dt]
    return x_pre, y_pre, rr, dt

In [9]:
lens_model_list = ['PlasmaExponentialSIE']
lensModel = LensModel(lens_model_list)
lensEquationSolver = LensEquationSolver(lensModel)
A = 2.00744300029693
B = 1.5887225542022732
C = 1.4308204791467736
psi0_plasma = (A*Dls/Ds/Dl*2*pi*c_light**2/omega**2*re*n0*r0 \
            *(u.rad**2).to('arcsec**2', equivalencies=u.dimensionless_angles()) ).decompose()

In [10]:
eta_arr = np.linspace(.05, .39, 100)
r_arr = np.linspace(.3, 3, 100)
x_arr = y_arr = r_arr*np.cos(np.pi/4)

In [ ]:
x_line_arr = []
y_line_arr = []
result = []
for x in x_arr:
    for i in eta_arr:
        kwargs = {'theta_E':theta_E_num.value,'eta':i, 'A':A, 'B':B, 'C':C ,'psi0_plasma':psi0_plasma.value,\
                        'theta_0':theta_0.value}
        kwargs_lens_list = [kwargs]
        x_image, y_image, mag, dt = image_position_3(x, x, i)
        tmp = [x, i, mag[0], dt[0]]
        result.append(tmp)
        print(x,i)

[ 0.59804034 -0.29882478]
[ 0.7587761 -0.2348826]
0.21213203435596426 0.05
[ 0.59246245 -0.30042381]
[ 0.76351238 -0.2323549 ]
0.21213203435596426 0.05343434343434344
[ 0.58693089 -0.30197174]
[ 0.76816577 -0.22982538]
0.21213203435596426 0.056868686868686874
[ 0.58144913 -0.30346824]
[ 0.77273575 -0.22729586]
0.21213203435596426 0.0603030303030303
[ 0.57602036 -0.3049131 ]
[ 0.77722208 -0.22476807]
0.21213203435596426 0.06373737373737375
[ 0.57064748 -0.30630617]
[ 0.78162477 -0.22224369]
0.21213203435596426 0.06717171717171717
[ 0.5653331  -0.30764741]
[ 0.78594407 -0.21972432]
0.21213203435596426 0.07060606060606062
[ 0.56007956 -0.30893687]
[ 0.79018044 -0.21721151]
0.21213203435596426 0.07404040404040405
[ 0.55488892 -0.31017466]
[ 0.79433454 -0.21470671]
0.21213203435596426 0.07747474747474747
[ 0.549763   -0.31136098]
[ 0.79840723 -0.21221133]
0.21213203435596426 0.08090909090909092
[ 0.54470334 -0.3124961 ]
[ 0.80239951 -0.20972667]
0.21213203435596426 0.08434343434343436
[ 0.5

[ 0.31888925 -0.29298744]
[ 0.97877156 -0.07527803]
0.21213203435596426 0.3590909090909091
[ 0.31746566 -0.29194165]
[ 0.98015645 -0.07429284]
0.21213203435596426 0.36252525252525253
[ 0.31606067 -0.2908846 ]
[ 0.98153319 -0.07331925]
0.21213203435596426 0.365959595959596
[ 0.31467394 -0.28981645]
[ 0.98290194 -0.07235709]
0.21213203435596426 0.36939393939393944
[ 0.31330513 -0.28873733]
[ 0.98426291 -0.07140618]
0.21213203435596426 0.37282828282828284
[ 0.3119539  -0.28764738]
[ 0.98561625 -0.07046637]
0.21213203435596426 0.3762626262626263
[ 0.31061994 -0.28654673]
[ 0.98696215 -0.06953747]
0.21213203435596426 0.3796969696969697
[ 0.30930291 -0.2854355 ]
[ 0.98830076 -0.06861934]
0.21213203435596426 0.38313131313131316
[ 0.30800253 -0.2843138 ]
[ 0.98963225 -0.0677118 ]
0.21213203435596426 0.3865656565656566
[ 0.30671847 -0.28318177]
[ 0.99095676 -0.06681471]
0.21213203435596426 0.39
[ 0.62176756 -0.27644688]
[ 0.77388749 -0.22159265]
0.23141676475196102 0.05
[ 0.61647928 -0.27783772

[ 0.36030538 -0.27950579]
[ 0.98084537 -0.08482349]
0.23141676475196102 0.3247474747474748
[ 0.35861241 -0.27862766]
[ 0.98236229 -0.08373656]
0.23141676475196102 0.3281818181818182
[ 0.35694204 -0.27773634]
[ 0.98386828 -0.08266196]
0.23141676475196102 0.33161616161616164
[ 0.35529387 -0.27683203]
[ 0.9853636  -0.08159953]
0.23141676475196102 0.3350505050505051
[ 0.35366746 -0.27591491]
[ 0.98684852 -0.0805491 ]
0.23141676475196102 0.3384848484848485
[ 0.35206243 -0.27498518]
[ 0.98832327 -0.07951052]
0.23141676475196102 0.34191919191919196
[ 0.35047838 -0.274043  ]
[ 0.9897881  -0.07848362]
0.23141676475196102 0.34535353535353536
[ 0.34891493 -0.27308854]
[ 0.99124324 -0.07746825]
0.23141676475196102 0.3487878787878788
[ 0.34737169 -0.27212198]
[ 0.99268891 -0.07646426]
0.23141676475196102 0.35222222222222227
[ 0.3458483  -0.27114347]
[ 0.99412532 -0.07547149]
0.23141676475196102 0.35565656565656567
[ 0.3443444  -0.27015316]
[ 0.9955527  -0.07448979]
0.23141676475196102 0.35909090909

[ 0.3969076  -0.26103024]
[ 0.98773181 -0.08929096]
0.25070149514795775 0.30414141414141416
[ 0.39501597 -0.26027661]
[ 0.98935949 -0.08817014]
0.25070149514795775 0.3075757575757576
[ 0.39314944 -0.25950859]
[ 0.99097402 -0.08706116]
0.25070149514795775 0.311010101010101
[ 0.39130754 -0.25872638]
[ 0.99257574 -0.08596388]
0.25070149514795775 0.31444444444444447
[ 0.38948983 -0.25793019]
[ 0.99416494 -0.0848782 ]
0.25070149514795775 0.3178787878787879
[ 0.38769589 -0.25712022]
[ 0.99574193 -0.08380397]
0.25070149514795775 0.32131313131313133
[ 0.38592529 -0.25629667]
[ 0.99730699 -0.08274108]
0.25070149514795775 0.3247474747474748
[ 0.38417761 -0.25545972]
[ 0.99886042 -0.08168939]
0.25070149514795775 0.3281818181818182
[ 0.38245244 -0.25460957]
[ 1.00040247 -0.08064879]
0.25070149514795775 0.33161616161616164
[ 0.38074939 -0.2537464 ]
[ 1.00193341 -0.07961914]
0.25070149514795775 0.3350505050505051
[ 0.37906807 -0.25287037]
[ 1.00345351 -0.07860032]
0.25070149514795775 0.3384848484848

[ 0.44102238 -0.24351776]
[ 0.9883951  -0.09598619]
0.2699862255439545 0.27323232323232327
[ 0.43884172 -0.24293615]
[ 0.9901962 -0.0948211]
0.2699862255439545 0.27666666666666667
[ 0.4366899  -0.24233846]
[ 0.99198056 -0.09366707]
0.2699862255439545 0.28010101010101013
[ 0.43456643 -0.24172492]
[ 0.99374857 -0.09252403]
0.2699862255439545 0.2835353535353536
[ 0.43247085 -0.24109575]
[ 0.99550064 -0.0913919 ]
0.2699862255439545 0.286969696969697
[ 0.43040267 -0.24045118]
[ 0.99723715 -0.0902706 ]
0.2699862255439545 0.29040404040404044
[ 0.42836143 -0.23979143]
[ 0.99895846 -0.08916006]
0.2699862255439545 0.2938383838383839
[ 0.42634668 -0.23911671]
[ 1.00066494 -0.0880602 ]
0.2699862255439545 0.2972727272727273
[ 0.42435797 -0.23842723]
[ 1.00235695 -0.08697094]
0.2699862255439545 0.3007070707070707
[ 0.42239484 -0.2377232 ]
[ 1.00403481 -0.0858922 ]
0.2699862255439545 0.30414141414141416
[ 0.42045688 -0.2370048 ]
[ 1.00569887 -0.08482388]
0.2699862255439545 0.3075757575757576
[ 0.4185

...

[1.43147777]
[1.84312198]
1.176368554155802 0.30414141414141416
[1.4293677]
[1.84514039]
1.176368554155802 0.3075757575757576
[1.42726507]
[1.84715189]
1.176368554155802 0.311010101010101
[1.4251699]
[1.84915648]
1.176368554155802 0.31444444444444447
[1.42308223]
[1.85115417]
1.176368554155802 0.3178787878787879
[1.42100207]
[1.85314496]
1.176368554155802 0.32131313131313133
[1.41892944]
[1.85512887]
1.176368554155802 0.3247474747474748
[1.41686438]
[1.85710591]
1.176368554155802 0.3281818181818182
[1.41480689]
[1.85907608]
1.176368554155802 0.33161616161616164
[1.41275699]
[1.8610394]
1.176368554155802 0.3350505050505051
[1.41071471]
[1.86299588]
1.176368554155802 0.3384848484848485
[1.40868005]
[1.86494553]
1.176368554155802 0.34191919191919196
[1.40665303]
[1.86688837]
1.176368554155802 0.34535353535353536
[1.40463367]
[1.86882441]
1.176368554155802 0.3487878787878788
[1.40262197]
[1.87075367]
1.176368554155802 0.35222222222222227
[1.40061795]
[1.87267616]
1.176368554155802 0.355656

[1.61879749]
[1.73854801]
1.2149380149477955 0.08434343434343436
[1.6163622]
[1.7409368]
1.2149380149477955 0.08777777777777779
[1.61392914]
[1.7433217]
1.2149380149477955 0.09121212121212122
[1.61149843]
[1.74570264]
1.2149380149477955 0.09464646464646465
[1.60907019]
[1.7480795]
1.2149380149477955 0.09808080808080809
[1.60664455]
[1.75045221]
1.2149380149477955 0.10151515151515153
[1.60422161]
[1.75282065]
1.2149380149477955 0.10494949494949496
[1.60180149]
[1.75518476]
1.2149380149477955 0.10838383838383839
[1.59938431]
[1.75754443]
1.2149380149477955 0.11181818181818183
[1.59697019]
[1.75989959]
1.2149380149477955 0.11525252525252526
[1.59455923]
[1.76225014]
1.2149380149477955 0.1186868686868687
[1.59215155]
[1.764596]
1.2149380149477955 0.12212121212121213
[1.58974726]
[1.7669371]
1.2149380149477955 0.1255555555555556
[1.58734647]
[1.76927334]
1.2149380149477955 0.128989898989899
[1.58494929]
[1.77160466]
1.2149380149477955 0.13242424242424244
[1.58255583]
[1.77393098]
1.21493801

[1.56474266]
[1.82931635]
1.2342227453437922 0.19080808080808082
[1.56243477]
[1.8315427]
1.2342227453437922 0.19424242424242427
[1.56013217]
[1.83376311]
1.2342227453437922 0.19767676767676767
[1.55783493]
[1.83597754]
1.2342227453437922 0.20111111111111113
[1.55554312]
[1.83818594]
1.2342227453437922 0.20454545454545459
[1.55325682]
[1.84038827]
1.2342227453437922 0.207979797979798
[1.5509761]
[1.84258451]
1.2342227453437922 0.21141414141414144
[1.54870103]
[1.84477461]
1.2342227453437922 0.21484848484848484
[1.54643169]
[1.84695853]
1.2342227453437922 0.2182828282828283
[1.54416814]
[1.84913625]
1.2342227453437922 0.22171717171717176
[1.54191045]
[1.85130773]
1.2342227453437922 0.22515151515151516
[1.53965868]
[1.85347294]
1.2342227453437922 0.22858585858585861
[1.5374129]
[1.85563186]
1.2342227453437922 0.23202020202020207
[1.53517317]
[1.85778445]
1.2342227453437922 0.23545454545454547
[1.53293955]
[1.85993069]
1.2342227453437922 0.23888888888888893
[1.53071209]
[1.86207056]
1.234

[1.50968402]
[1.92031733]
1.253507475739789 0.3075757575757576
[1.50759268]
[1.922326]
1.253507475739789 0.311010101010101
[1.50550827]
[1.9243282]
1.253507475739789 0.31444444444444447
[1.50343081]
[1.92632392]
1.253507475739789 0.3178787878787879
[1.50136032]
[1.92831318]
1.253507475739789 0.32131313131313133
[1.49929684]
[1.93029598]
1.253507475739789 0.3247474747474748
[1.49724037]
[1.93227232]
1.253507475739789 0.3281818181818182
[1.49519096]
[1.9342422]
1.253507475739789 0.33161616161616164
[1.4931486]
[1.93620564]
1.253507475739789 0.3350505050505051
[1.49111334]
[1.93816264]
1.253507475739789 0.3384848484848485
[1.48908518]
[1.94011321]
1.253507475739789 0.34191919191919196
[1.48706414]
[1.94205735]
1.253507475739789 0.34535353535353536
[1.48505023]
[1.94399508]
1.253507475739789 0.3487878787878788
[1.48304348]
[1.94592641]
1.253507475739789 0.35222222222222227
[1.48104389]
[1.94785134]
1.253507475739789 0.35565656565656567
[1.47905149]
[1.94976989]
1.253507475739789 0.35909090

[1.67575462]
[1.83614938]
1.2920769365317823 0.11525252525252526
[1.67338039]
[1.83847008]
1.2920769365317823 0.1186868686868687
[1.67100915]
[1.84078653]
1.2920769365317823 0.12212121212121213
[1.66864098]
[1.84309866]
1.2920769365317823 0.1255555555555556
[1.66627599]
[1.8454064]
1.2920769365317823 0.128989898989899
[1.66391429]
[1.84770967]
1.2920769365317823 0.13242424242424244
[1.66155596]
[1.85000841]
1.2920769365317823 0.13585858585858587
[1.65920111]
[1.85230253]
1.2920769365317823 0.1392929292929293
[1.65684983]
[1.85459197]
1.2920769365317823 0.14272727272727276
[1.65450222]
[1.85687667]
1.2920769365317823 0.14616161616161616
[1.65215837]
[1.85915656]
1.2920769365317823 0.14959595959595962
[1.64981837]
[1.86143157]
1.2920769365317823 0.15303030303030304
[1.64748232]
[1.86370164]
1.2920769365317823 0.15646464646464647
[1.6451503]
[1.86596672]
1.2920769365317823 0.15989898989898993
[1.64282241]
[1.86822673]
1.2920769365317823 0.16333333333333333
[1.64049872]
[1.87048163]
1.2920

In [ ]:
end_total = time.time()
print('total time: ',(end_total-start_total)/60.,' minutes')